In [1]:
import re

import kagglehub
import keras
import keras_hub
import keras_tuner
import pandas as pd
import pkuseg
import tensorflow as tf
from datasets import Dataset
from pypinyin import lazy_pinyin
from keras.layers import (
    LSTM,
    Dense,
    Embedding,
    Input,
    StringLookup,
    TextVectorization,
)
from keras.models import Model


path = kagglehub.dataset_download("noxmoon/chinese-official-daily-news-since-2016")

print("Path to dataset files:", path)

100%|██████████| 8.43M/8.43M [00:00<00:00, 11.7MB/s]

Extracting files...


Path to dataset files: /Users/zhongjie/.cache/kagglehub/datasets/noxmoon/chinese-official-daily-news-since-2016/versions/1


# Création du corpus

In [2]:
dataset = pd.read_csv(path+"/chinese_news.csv")
# Print dataset information
print("Dataset information:")
print(dataset.info())
# Print dataset head
print("Dataset head:")
print(dataset.head())

Dataset information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20738 entries, 0 to 20737
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      20738 non-null  object
 1   tag       20738 non-null  object
 2   headline  20738 non-null  object
 3   content   20631 non-null  object
dtypes: object(4)
memory usage: 648.2+ KB
None
Dataset head:
         date   tag                                           headline  \
0  2016-01-01  详细全文  陆军领导机构火箭军战略支援部队成立大会在京举行 习近平向中国人民解放军陆军火箭军战略支援部队...   
1  2016-01-01  详细全文                             中央军委印发《关于深化国防和军队改革的意见》   
2  2016-01-01  详细全文                           《习近平关于严明党的纪律和规矩论述摘编》出版发行   
3  2016-01-01  详细全文                                 以实际行动向党中央看齐 向高标准努力   
4  2016-01-01  详细全文                                 【年终特稿】关键之年 改革挺进深水区   

                                             content  
0  中国人民解放军陆军领导机构、中国人民解放军火箭军、中国人民解放军战略支援部队成立大会2015...  
1  经中央军委主席习近平批准，中央军委近日印发了《关

In [3]:
# Prétraitement de content (suppression des caractères non chinois, normalisation des espaces)
def clean_content(text):
    if not isinstance(text, str):
        return ""
    
    # Garder les caractères chinois et ponctuation chinoise
    text = re.sub(r"[^\u4e00-\u9fff\u3000-\u303F\uff00-\uffef]", "", text)
    
    # Normaliser les espaces (rare, mais au cas où)
    text = text.replace(" ", "").strip()

    return text

# Appliquer le prétraitement à la colonne 'content'
dataset['cleaned_content'] = dataset['content'].apply(clean_content)

# Afficher les 5 premières lignes du DataFrame après le prétraitement
print("Dataset after preprocessing:")
print(dataset[['content', 'cleaned_content']].head())

seg = pkuseg.pkuseg()
dataset["tokens"] = dataset["cleaned_content"].apply(lambda x: seg.cut(x))

# Aperçu
print(dataset["tokens"].head())

Dataset after preprocessing:
                                             content  \
0  中国人民解放军陆军领导机构、中国人民解放军火箭军、中国人民解放军战略支援部队成立大会2015...   
1  经中央军委主席习近平批准，中央军委近日印发了《关于深化国防和军队改革的意见》。\n《意见》强...   
2  由中共中央纪律检查委员会、中共中央文献研究室编辑的《习近平关于严明党的纪律和规矩论述摘编》一...   
3  广大党员干部正在积极学习习近平总书记在中央政治局专题民主生活会上的重要讲话。大家纷纷表示要把...   
4  刚刚过去的2015年，是全面深化改革的关键之年。改革集中发力在制约经济社会发展的深层次矛盾，...   

                                     cleaned_content  
0  中国人民解放军陆军领导机构、中国人民解放军火箭军、中国人民解放军战略支援部队成立大会年月日在...  
1  经中央军委主席习近平批准，中央军委近日印发了《关于深化国防和军队改革的意见》。《意见》强调，...  
2  由中共中央纪律检查委员会、中共中央文献研究室编辑的《习近平关于严明党的纪律和规矩论述摘编》一...  
3  广大党员干部正在积极学习习近平总书记在中央政治局专题民主生活会上的重要讲话。大家纷纷表示要把...  
4  刚刚过去的年，是全面深化改革的关键之年。改革集中发力在制约经济社会发展的深层次矛盾，集中发力...  
0    [中国, 人民, 解放军, 陆军, 领导, 机构, 、, 中国, 人民, 解放军, 火箭军,...
1    [经, 中央军委, 主席, 习近平, 批准, ，, 中央军委, 近日, 印发, 了, 《, ...
2    [由, 中共中央, 纪律, 检查, 委员会, 、, 中共中央, 文献, 研究室, 编辑, 的...
3    [广大, 党员, 干部, 正在, 积极, 学习, 习近平, 总书记, 在, 中央, 政治局,...
4    [刚刚, 过去, 的, 年, ，, 是, 全面, 深化, 改革, 的, 关键, 之, 年, ...
Name: tokens, dtype: object


In [4]:
# convert the content column to pinyin
t9_map = {
    "@": "1", ".": "1", ":": "1",
    "a": "2", "b": "2", "c": "2",
    "d": "3", "e": "3", "f": "3",
    "g": "4", "h": "4", "i": "4",
    "j": "5", "k": "5", "l": "5",
    "m": "6", "n": "6", "o": "6",
    "p": "7", "q": "7", "r": "7", "s": "7",
    "t": "8", "u": "8", "v": "8",
    "w": "9", "x": "9", "y": "9", "z": "9",
    "1": "1", "2": "2", "3": "3", "4": "4",
    "5": "5", "6": "6", "7": "7", "8": "8",
    "9": "9", "0": "0", " ": "0",
    "。":"。", "，":"，", "？":"？", "！":"！",
}

# Fonction pour convertir une chaîne de caractères en code T9
def pinyin_to_t9(text):
    t9_code = ""
    # print(text)
    if pd.isna(text):
        return text
    for char in text:
        if char.lower() in t9_map:
            t9_code += t9_map[char.lower()]
        else:
            t9_code += char  # Conserver les caractères non mappés tels quels
    return t9_code

def generer_sequence_contextuelle(row):
    tokens = eval(row["tokens"]) if isinstance(row["tokens"], str) else row["tokens"]
    sequence = []
    for token in tokens:
        if not isinstance(token, str) or not re.search(r'[\u4e00-\u9fff]', token):
            continue
        for char, py in zip(token, lazy_pinyin(token)):
            t9 = pinyin_to_t9(py)
            sequence.append(f"{char}|{py}|{t9}")
    return ' '.join(sequence)

dataset["char_pinyin_t9_sequence"] = dataset.apply(generer_sequence_contextuelle, axis=1)

# Sauvegarder le fichier
dataset[["char_pinyin_t9_sequence"]].to_csv("sequences_char_pinyin_t9.csv", index=False)

# Afficher les 5 premières lignes du DataFrame après le prétraitement
print("Dataset after generating sequences:")
print(dataset[['content', 'char_pinyin_t9_sequence']].head())

Dataset after generating sequences:
                                             content  \
0  中国人民解放军陆军领导机构、中国人民解放军火箭军、中国人民解放军战略支援部队成立大会2015...   
1  经中央军委主席习近平批准，中央军委近日印发了《关于深化国防和军队改革的意见》。\n《意见》强...   
2  由中共中央纪律检查委员会、中共中央文献研究室编辑的《习近平关于严明党的纪律和规矩论述摘编》一...   
3  广大党员干部正在积极学习习近平总书记在中央政治局专题民主生活会上的重要讲话。大家纷纷表示要把...   
4  刚刚过去的2015年，是全面深化改革的关键之年。改革集中发力在制约经济社会发展的深层次矛盾，...   

                             char_pinyin_t9_sequence  
0  中|zhong|94664 国|guo|486 人|ren|736 民|min|646 解|...  
1  经|jing|5464 中|zhong|94664 央|yang|9264 军|jun|58...  
2  由|you|968 中|zhong|94664 共|gong|4664 中|zhong|94...  
3  广|guang|48264 大|da|32 党|dang|3264 员|yuan|9826 ...  
4  刚|gang|4264 刚|gang|4264 过|guo|486 去|qu|78 的|de...  


# Création du dataset pour le modèle

In [5]:
# Transformer en séquences complètes
input_t9_sequences = []
target_char_sequences = []

for seq in dataset["char_pinyin_t9_sequence"]:
    triplets = seq.strip().split(" ")
    t9_seq = []
    char_seq = []
    
    # Extraire les paires char|T9 pour chaque phrase
    for triplet in triplets:
        parts = triplet.split("|")
        if len(parts) == 3:
            char, _, t9 = parts
            char_seq.append(char)
            t9_seq.append(t9)
    # Ajouter les séquences T9 et caractères
    input_t9_sequences.append(" ".join(t9_seq))
    target_char_sequences.append("".join(char_seq))

# Créer un DataFrame
df_sequences = pd.DataFrame({
    "input_t9_sequence": input_t9_sequences,
    "target_char_sequence": target_char_sequences
})

In [6]:
print("DataFrame sequences:")
print(df_sequences.head())

DataFrame sequences:
                                   input_t9_sequence  \
0  94664 486 736 646 543 3264 586 58 586 5464 326...   
1  5464 94664 9264 586 934 948 94 94 546 7464 74 ...   
2  968 94664 4664 94664 9264 54 58 5426 242 934 9...   
3  48264 32 3264 9826 426 28 94364 924 54 54 983 ...   
4  4264 4264 486 78 33 6426 744 7826 6426 7436 48...   

                                target_char_sequence  
0  中国人民解放军陆军领导机构中国人民解放军火箭军中国人民解放军战略支援部队成立大会年月日在八一...  
1  经中央军委主席习近平批准中央军委近日印发了关于深化国防和军队改革的意见意见强调党的十八大以来...  
2  由中共中央纪律检查委员会中共中央文献研究室编辑的习近平关于严明党的纪律和规矩论述摘编一书近日...  
3  广大党员干部正在积极学习习近平总书记在中央政治局专题民主生活会上的重要讲话大家纷纷表示要把践...  
4  刚刚过去的年是全面深化改革的关键之年改革集中发力在制约经济社会发展的深层次矛盾集中发力在妨碍...  


In [ ]:
# Utiliser tf.data.Dataset
tf_dataset = tf.data.Dataset.from_tensor_slices((input_t9_sequences, target_char_sequences)).prefetch(tf.data.AUTOTUNE)
tf_dataset.take(1).get_single_element()

(<tf.Tensor: shape=(), dtype=string, numpy=b'94664 486 736 646 543 3264 586 58 586 5464 326 54 468 94664 486 736 646 543 3264 586 486 5426 586 94664 486 736 646 543 3264 586 9426 583 944 9826 28 384 24364 54 32 484 6426 983 74 924 22 94 32 568 5664 94664 58 9464 94664 4664 94664 9264 9664 748 54 486 542 948 94 94664 9264 586 934 948 94 94 546 7464 94264 58 586 486 5426 586 9426 583 944 9826 28 384 7468 98 586 74 2464 944 986 24 324 2426 3264 94664 9264 43 94664 9264 586 934 94264 8664 944 636 94264 7826 586 28 384 944 94 73 543 948 43 74264 3426 926 5426 244 94 3264 924 946 9464 744 942 33 74264 586 68 2426 934 946 5464 7436 78 4826 243 946 9464 744 942 586 744 9426 583 3264 9436 7826 6426 744 744 424 43 74264 586 9426 583 5426 3464 28 94 968 94664 486 83 73 74264 586 944 58 744 53 8464 2664 3264 43 736 646 9426 4826 94664 744 58 9464 3264 43 736 646 38 98 33 7436 74364 744 6464 934 744 9426 94664 486 6364 74264 586 6364 986 248 946 33 4364 32 33 4664 9426 942 98 744 24364 54 32 484 52

## Encoder les données pour Keras

In [ ]:
# TextVectorization
input_tv = keras.layers.TextVectorization(output_mode='int',
                                          split='whitespace',
                                          standardize=None,
                                          ragged=True,)

target_tv = keras.layers.TextVectorization(output_mode='int',
                                           split='character',
                                           standardize=None,
                                           ragged=True,)
t9_ds = tf_dataset.map(lambda t9, target: t9)
target_ds = tf_dataset.map(lambda t9, target: target)
input_tv.adapt(t9_ds)
target_tv.adapt(target_ds)

2025-04-10 23:20:10.388463: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-04-10 23:20:30.618885: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [16]:
@tf.function
def transform_ds(t9, target):
    vectorized_t9 = input_tv(t9)
    vectorized_target = target_tv(target)
    return (vectorized_t9, vectorized_target)

transformed_tf_dataset = tf_dataset.map(transform_ds, num_parallel_calls=20)

In [18]:
transformed_tf_dataset.padded_batch(3).take(1).get_single_element()

(<tf.Tensor: shape=(3, 1293), dtype=int64, numpy=
 array([[10,  7, 30, ..., 41,  9, 20],
        [23, 10, 50, ...,  0,  0,  0],
        [43, 10, 19, ...,  0,  0,  0]])>,
 <tf.Tensor: shape=(3, 1293), dtype=int64, numpy=
 array([[  4,   3,   8, ...,  47,   9,  10],
        [ 53,   4,  84, ...,   0,   0,   0],
        [339,   4,  48, ...,   0,   0,   0]])>)

## Split train-valid-test

In [25]:
c = transformed_tf_dataset.reduce(0, lambda x,_:x+1).numpy()

shuffled_ds = transformed_tf_dataset.shuffle(buffer_size=c, seed=42)

train_size = c * 80 // 100
test_size = c * 10 // 100
val_size = c - train_size - test_size

ds_train = shuffled_ds.take(train_size).prefetch(tf.data.AUTOTUNE)
ds_val = shuffled_ds.skip(train_size).take(val_size).prefetch(tf.data.AUTOTUNE)
ds_test = shuffled_ds.skip(train_size+val_size).take(test_size).prefetch(tf.data.AUTOTUNE)

print("Taille du train :", ds_train.cardinality().numpy())
print("Taille du validation :", ds_val.cardinality().numpy())
print("Taille du test :", ds_test.cardinality().numpy())

Taille du train : 16590
Taille du validation : 2075
Taille du test : 2073


# Modèle

Sogou T9 est une méthode d’entrée intelligente qui :

- Prend des séquences numériques (ex. : "94664 486" pour "zhong guo").
- Génère des séquences de caractères chinois (ex. : "中国").
- Utilise le contexte (mots précédents) pour désambiguïser les prédictions.
- Est optimisé pour la vitesse et la précision, souvent avec des modèles entraînés sur de vastes corpus.

Pour reproduire cela, il faut utiliser un modèle seq2seq avec un encodeur-décodeur (2 entrées) :

- Encodeur : Lit la séquence T9 et la compresse en une représentation contextuelle.
- Décodeur : Génère la séquence de caractères chinois à partir de cette représentation.

[Functional API](https://keras.io/guides/functional_api/)

# Génération